# Inject sources into multiple images, then align them

Contact author: Jeff Carlin

Date last verified to run: Mon Apr 29 2024

RSP environment version: Weekly 2024_16

**Summary:**
A demo of how to inject sources into a set of `calexp` images, then "warp" those images to a common WCS so that they are aligned.

Import packages and then instantiate a butler for DP0.2.

In [ ]:
import matplotlib.pyplot as plt
import os
import astropy.units as u
from astropy.coordinates import SkyCoord

from lsst.daf.butler import Butler
from lsst.daf.butler.registry import ConflictingDefinitionError
import lsst.afw.display as afwDisplay
from lsst.source.injection import ingest_injection_catalog, generate_injection_catalog
from lsst.source.injection import VisitInjectConfig, VisitInjectTask
import lsst.sphgeom
from lsst.pipe.tasks.registerImage import RegisterConfig, RegisterTask

afwDisplay.setDefaultBackend('matplotlib')
plt.style.use('tableau-colorblind10')

In [ ]:
butler_config = 'dp02'
collections = '2.2i/runs/DP0.2'
butler = Butler(butler_config, collections=collections)

### Find calexps overlapping a given position on the sky:

In [ ]:
ra_known_rrl = 62.1479031
dec_known_rrl = -35.799138

In [ ]:
level = 20  # the resolution of the HTM grid
pixelization = lsst.sphgeom.HtmPixelization(level)

In [ ]:
htm_id = pixelization.index(
    lsst.sphgeom.UnitVector3d(
        lsst.sphgeom.LonLat.fromDegrees(ra_known_rrl, dec_known_rrl)
    )
)

In [ ]:
circle = pixelization.triangle(htm_id).getBoundingCircle()
scale = circle.getOpeningAngle().asDegrees()*3600.
level = pixelization.getLevel()
print(f'HTM ID={htm_id} at level={level} is bounded by a circle of radius ~{scale:0.2f} arcsec.')

In [ ]:
datasetRefs = butler.registry.queryDatasets("calexp", htm20=htm_id,
                                            where="band = 'i'")

datasetRefs_list = []
for i, ref in enumerate(datasetRefs):
    datasetRefs_list.append(ref)

print(f"Found {len(list(datasetRefs))} calexps")

### Extract 3 calexp images to inject sources into

In [ ]:
dataId_i1 = datasetRefs_list[1].dataId
dataId_i2 = datasetRefs_list[6].dataId
dataId_i3 = datasetRefs_list[7].dataId

print(f"{dataId_i1 = }")
print(f"{dataId_i2 = }")
print(f"{dataId_i3 = }")

In [ ]:
calexp_i1 = butler.get('calexp', dataId=dataId_i1)
calexp_i2 = butler.get('calexp', dataId=dataId_i2)
calexp_i3 = butler.get('calexp', dataId=dataId_i3)

Extract coordinates, the WCS, and the bounding box for each `calexp`.

In [ ]:
wcs1 = calexp_i1.getWcs()
bbox1 = calexp_i1.getBBox()
print('bounding box: ', bbox1)

boxcen1 = bbox1.getCenter()
cen1 = wcs1.pixelToSky(boxcen1)
sc_cen1 = SkyCoord(ra=cen1[0].asDegrees()*u.deg, dec=cen1[1].asDegrees()*u.deg)

print(sc_cen1)

wcs2 = calexp_i2.getWcs()
bbox2 = calexp_i2.getBBox()
print('bounding box: ', bbox2)

boxcen2 = bbox2.getCenter()
cen2 = wcs2.pixelToSky(boxcen2)
sc_cen2 = SkyCoord(ra=cen2[0].asDegrees()*u.deg, dec=cen2[1].asDegrees()*u.deg)

print(sc_cen2)

wcs3 = calexp_i3.getWcs()
bbox3 = calexp_i3.getBBox()
print('bounding box: ', bbox3)

boxcen3 = bbox3.getCenter()
cen3 = wcs3.pixelToSky(boxcen3)
sc_cen3 = SkyCoord(ra=cen3[0].asDegrees()*u.deg, dec=cen3[1].asDegrees()*u.deg)

print(sc_cen3)

### Create a catalog of sources to inject

In [ ]:
inject_size = 3/60  # in degrees

This will generate 21 "Sersic" type sources (i.e., "galaxies"), all with the same magnitude (mag), Sersic index (n), ellipticity (q), position angle (beta), and half-light radius.

In [ ]:
my_injection_catalog_galaxies = generate_injection_catalog(
    ra_lim=[sc_cen1.ra.value-inject_size, sc_cen1.ra.value+inject_size],
    dec_lim=[sc_cen1.dec.value-inject_size, sc_cen1.dec.value+inject_size],
    number=21,
    seed='3210',
    source_type="Sersic",
    mag=[15.0],
    n=[1],
    q=[0.5],
    beta=[31.0],
    half_light_radius=[15.0],
)

In [ ]:
inject_cat = my_injection_catalog_galaxies

### Ingest the catalog into a butler

In [ ]:
# Get username.
user = os.getenv("USER")

INJECTION_CATALOG_COLLECTION = f"u/{user}/injection_inputs21_contrib"

# Instantiate a writeable butler.
writeable_butler = Butler(butler_config, writeable=True)

In [ ]:
try:
    my_injected_datasetRefs = ingest_injection_catalog(
        writeable_butler=writeable_butler,
        table=inject_cat,
        band="i",
        output_collection=INJECTION_CATALOG_COLLECTION,
    )
except ConflictingDefinitionError:
    print(f"Found an existing collection named INJECTION_CATALOG_COLLECTION={INJECTION_CATALOG_COLLECTION}.")
    print("\nNOTE THAT IF YOU SEE THIS MESSAGE, YOUR CATALOG WAS NOT INGESTED."\
          "\nYou may either continue with the pre-existing catalog, or choose a new"\
          " name and re-run the previous cell and this one to ingest a new catalog.")

### Inject the sources into all 3 calexp images

In [ ]:
psf1 = calexp_i1.getPsf()
photo_calib1 = calexp_i1.getPhotoCalib()
wcs1 = calexp_i1.getWcs()

psf2 = calexp_i2.getPsf()
photo_calib2 = calexp_i2.getPhotoCalib()
wcs2 = calexp_i2.getWcs()

psf3 = calexp_i3.getPsf()
photo_calib3 = calexp_i3.getPhotoCalib()
wcs3 = calexp_i3.getWcs()

In [ ]:
# Load the input injection catalogs from the butler.
injection_refs = butler.registry.queryDatasets(
    "injection_catalog",
    band="i",
    collections=INJECTION_CATALOG_COLLECTION,
)
injection_catalogs = [
    butler.get(injection_ref) for injection_ref in injection_refs
]

Initialize the injection task, then run it on each of the 3 `calexps`.

In [ ]:
inject_config = VisitInjectConfig()
inject_task = VisitInjectTask(config=inject_config)

In [ ]:
injected_output1 = inject_task.run(
    injection_catalogs=injection_catalogs,
    input_exposure=calexp_i1.clone(),
    psf=psf1,
    photo_calib=photo_calib1,
    wcs=wcs1,
)
injected_exposure1 = injected_output1.output_exposure
injected_catalog1 = injected_output1.output_catalog

In [ ]:
injected_output2 = inject_task.run(
    injection_catalogs=injection_catalogs,
    input_exposure=calexp_i2.clone(),
    psf=psf2,
    photo_calib=photo_calib2,
    wcs=wcs2,
)
injected_exposure2 = injected_output2.output_exposure
injected_catalog2 = injected_output2.output_catalog

In [ ]:
injected_output3 = inject_task.run(
    injection_catalogs=injection_catalogs,
    input_exposure=calexp_i3.clone(),
    psf=psf3,
    photo_calib=photo_calib3,
    wcs=wcs3,
)
injected_exposure3 = injected_output3.output_exposure
injected_catalog3 = injected_output3.output_catalog

### Display the source-injected images

In [ ]:
plot_injected_calexp1 = injected_exposure1.clone()
plot_injected_calexp2 = injected_exposure2.clone()
plot_injected_calexp3 = injected_exposure3.clone()

fig, ax = plt.subplots(1, 3, figsize=(10, 6), dpi=150)

plt.sca(ax[0])
display0 = afwDisplay.Display(frame=fig)
# display0.scale('linear', 'zscale')
display0.scale('linear', min=-20, max=150)
display0.mtv(plot_injected_calexp1.image)
plt.title('injected_calexp image1')

plt.sca(ax[1])
display1 = afwDisplay.Display(frame=fig)
# display1.scale('linear', 'zscale')
display1.scale('linear', min=-20, max=150)
display1.mtv(plot_injected_calexp2.image)
plt.title('injected_calexp image2')

plt.sca(ax[2])
display2 = afwDisplay.Display(frame=fig)
# display1.scale('linear', 'zscale')
display2.scale('linear', min=-20, max=150)
display2.mtv(plot_injected_calexp3.image)
plt.title('injected_calexp image3')

plt.tight_layout()
plt.show()

### Warp the images to match the WCS of the first one, then display them

In the above image, you can see that the `calexp` images have different rotation on the sky. The following cells will "warp" the images to a common sky orientation. This is done using the `RegisterTask` and giving it the first `calexp` as the reference image.

In [ ]:
def warp_img(ref_img, img_to_warp, ref_wcs, wcs_to_warp):

    config = RegisterConfig()
    task = RegisterTask(name="register", config=config)
    warpedExp = task.warpExposure(img_to_warp, wcs_to_warp, ref_wcs,
                                  ref_img.getBBox())

    return warpedExp

In [ ]:
img_warped2 = warp_img(injected_exposure1, injected_exposure2, wcs1, wcs2)
img_warped3 = warp_img(injected_exposure1, injected_exposure3, wcs1, wcs3)

In [ ]:
plot_warped_calexp2 = img_warped2.clone()
plot_warped_calexp3 = img_warped3.clone()

fig, ax = plt.subplots(1, 3, figsize=(10, 6), dpi=150)

plt.sca(ax[0])
display0 = afwDisplay.Display(frame=fig)
# display0.scale('linear', 'zscale')
display0.scale('linear', min=-20, max=150)
display0.mtv(plot_injected_calexp1.image)
plt.title('injected_calexp image1')

plt.sca(ax[1])
display1 = afwDisplay.Display(frame=fig)
# display1.scale('linear', 'zscale')
display1.scale('linear', min=-20, max=150)
display1.mtv(plot_warped_calexp2.image)
plt.title('warped calexp image2')

plt.sca(ax[2])
display2 = afwDisplay.Display(frame=fig)
# display1.scale('linear', 'zscale')
display2.scale('linear', min=-20, max=150)
display2.mtv(plot_warped_calexp3.image)
plt.title('warped calexp image3')

plt.tight_layout()
plt.show()

Hooray -- they all seem to line up!

### Confirm that they are aligned by summing them

Note that by default this will only add pixels that are present in all three images.

In [ ]:
summed_image = plot_injected_calexp1.clone().image
summed_image += plot_warped_calexp2.image
summed_image += plot_warped_calexp3.image

fig, ax = plt.subplots(1, 2, figsize=(8, 4), dpi=150)

plt.sca(ax[0])
display0 = afwDisplay.Display(frame=fig)
# display0.scale('linear', 'zscale')
display0.scale('linear', min=-20, max=150)
display0.mtv(plot_injected_calexp1.image)

plt.sca(ax[1])
display1 = afwDisplay.Display(frame=fig)
# display1.scale('linear', 'zscale')
display1.scale('linear', min=-20, max=150)
display1.mtv(summed_image)

plt.tight_layout()
plt.show()